<a href="https://colab.research.google.com/github/bernardo607/geocoding_project/blob/main/proyecto_geolocalizacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Geolocalización de clientes**
### **Bernardo Seguí**

### El dataset original ha sido modificado levemente para resguardar los datos sensibles y se ha limitado la cantidad de registros a 1000, siendo realmente algunas decenas de miles.

### El **OBJETIVO** de este trabajo es geolocalizar direcciones y comparar los servicios de Google y Nominatim (OSM), utilizando las librerías pandas, numpy y matplotlib.

### **IMPORTANTE PARA CORRER SCRIPT:**

#### El dataset original y el archivo preprocesado de geolocalización se importan directamente del repositorio de github detallado en el código.
#### En caso de reutilizar el código con nuevos datases, deben cargarse las 'keys' personales para conectarse con las APIs de geocodificación.

### Importo librerías y cargo dataset

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from geopy.extra.rate_limiter import RateLimiter #para agregar un delay entre solicitudes a las APIs
from geopy.geocoders import GoogleV3 #para geolocalizar con Google
from geopy.geocoders import Nominatim #para geolocalizar con OpenStreetMap
from geopy import distance #para calcular distancias versus coordenadas reales
from bokeh.plotting import figure, show, output_notebook #para graficar clientes en el mapa
from bokeh.tile_providers import CARTODBPOSITRON,get_provider,Vendors #para tener el tile del mapa
from collections import Counter #para contar palabras en direcciones fallidas
from google.colab import files #permite cargar los archivos adjuntos

In [ ]:
#importo repositorio de github con dataset
# !git clone -l -s https://github.com/bernardo607/geocoding_project.git geocoding_files
# %cd geocoding_files
# !ls

In [ ]:
df=pd.read_excel("data_raw_1000.xlsx")
df.head()

### Uppercase de nombres de columnas y renombro columnas

In [ ]:
#nombres de columnas con uppercase
df.columns = df.columns.str.upper()
df.columns

In [ ]:
#renombro algunas columnas 
df.rename(columns={'DWDLOCALIDAD.NOMBRE':'LOCALIDAD','DWDPAIS.NOMBRE':'PAIS','DWDPARTIDO.NOMBRE':'PARTIDO','DWDPROVINCIA.NOMBRE':'PROVINCIA'},inplace=True)

In [ ]:
#paso a uppercase los registros en columnas relevantes que fomarán la dirección a geolocalizar
df['DIRECCION']=df['DIRECCION'].str.upper()
df['LOCALIDAD'] = df['LOCALIDAD'].str.upper()
df['PARTIDO'] = df['PARTIDO'].str.upper()
df['PROVINCIA'] = df['PROVINCIA'].str.upper()
df['PAIS'] = df['PAIS'].str.upper()

In [ ]:
df.head()

### Limpio y concateno los datos que conformarán la dirección enviada a geolocalizar

In [ ]:
#Para cada columna, elimino los espacios blancos a izquierda y derecha de cada registro
df['DIRECCION'] = df['DIRECCION'].map(lambda x: x.replace(',','.').replace('S/N','SN').replace('KMS','KM').strip())
df['LOCALIDAD'] = df['LOCALIDAD'].str.strip()
df['PARTIDO'] = df['PARTIDO'].str.strip()
df['PROVINCIA'] = df['PROVINCIA'].str.strip()
df['PAIS'] = df['PAIS'].str.strip()

In [ ]:
#concateno los elementos que formaran la direccion
df['DIRECCION_CONCAT'] = df['DIRECCION'].astype(str) + ',' + df['LOCALIDAD'].astype(str) + ',' + df['PARTIDO'].astype(str) + ',' +\
df['PROVINCIA'].astype(str) + ',' +df['PAIS'].astype(str)

In [ ]:
#veo como queda la direccion armada
df['DIRECCION_CONCAT']

### Geolocalización con Google

Este bloque conecta con la API de Google y requiere una KEY propia.
Esto fue procesado previamente, se toma el archivo geocodificado de github. Descomentar en caso de querer correr con key propia.

In [ ]:
'''
locator_google = GoogleV3(api_key='********') #completar API key de Google

geocode_google = RateLimiter(locator_google.geocode, min_delay_seconds=0.5) #delay entre paquetes para geocodificar

df['LOCATION_GOOGLE'] = df['DIRECCION_CONCAT'].apply(geocode_google, region="ar") #crea columna location

df['PUNTO_GOOGLE'] = df['LOCATION_GOOGLE'].apply(lambda loc: tuple(loc.point) if loc else None) #crea longitude, latitude y altitude de columna location

df[['LATITUDE_GOOGLE', 'LONGITUDE_GOOGLE', 'ALTITUDE_GOOGLE']] = pd.DataFrame(df['PUNTO_GOOGLE'].tolist(), index=df.index) #divide columna en latitude, longitude y altitude
'''

In [ ]:
'''
#guarda archivo csv, puede luego ser procesado con QGIS o algún software similar
#en este caso se preprocesó y se levanta del repo de github, "geocoded_Google_OSM.csv"
df.to_csv(r"********", index = False)
'''

### Geolocalización con OpenStreetMaps

Este bloque conecta con la API de OSM y requiere una KEY propia.
Esto fue procesado previamente, se toma el archivo geocodificado de github. Descomentar en caso de querer correr con key propia.

In [ ]:
'''
locator_osm = Nominatim(user_agent='********') #completar API key de OSM

geocode_osm = RateLimiter(locator_osm.geocode, min_delay_seconds=1) #delay entre paquetes para geocodificar

df['LOCATION_OSM'] = df['DIRECCION_CONCAT'].apply(locator_osm.geocode, country_codes="ar", language='es') #crea columna location

df['PUNTO_OSM'] = df['LOCATION_OSM'].apply(lambda loc: tuple(loc.point) if loc else None) #crea longitude, latitude y altitude de columna location

df[['LATITUDE_OSM', 'LONGITUDE_OSM', 'ALTITUDE_OSM']] = pd.DataFrame(df['PUNTO_OSM'].tolist(), index=df.index) #divide columna en latitude, longitude y altitude
'''


In [ ]:
'''
#guarda archivo csv, puede luego ser procesado con QGIS o algún software similar
#en este caso se preprocesó y se levanta del repo de github, "geocoded_Google_OSM.csv"
df.to_csv(r"********", index = False)
'''

### Análisis de direcciones no resueltas

Leo el archivo preprocesado "geocoded_Google_OSM.csv"


In [ ]:
df=pd.read_csv("geocoded_Google_OSM.csv")

In [ ]:
#direcciones no resueltas Google
no_resuelto_google= pd.isna(df['PUNTO_GOOGLE']).sum()
nro_direcciones= len(df.index)
print(f'Google API: {no_resuelto_google} direcciones no resueltas sobre un total de {nro_direcciones}')
#direcciones no resueltas OSM
no_resuelto_osm= pd.isna(df['PUNTO_OSM']).sum()
nro_direcciones= len(df.index)
print(f'OpenStreetMaps: {no_resuelto_osm} direcciones no resueltas sobre un total de {nro_direcciones}')

#### Casos no resueltos con Google

Tomo las direcciones no resueltas y analizo las palabras más comunes para identificar patrones.

In [ ]:
df_googlefail=df[df['PUNTO_GOOGLE'].isna()]
df_googlefail['DIRECCION']

In [ ]:
results_google = Counter()
df_googlefail['DIRECCION'].str.split().apply(results_google.update)

for key, value in results_google.most_common():
    print(f'Frecuencia: {value}, Palabra: {key} ')

Usando Google se ven sólo 6 casos no resueltos con lo cual es difícil encontrar patrones.

#### Casos no resueltos con OSM

Tomo las direcciones no resueltas y analizo las palabras más comunes para identificar patrones.

In [ ]:
df_osmfail=df[df['PUNTO_OSM'].isna()]

results_osm = Counter()
df_osmfail['DIRECCION'].str.split().apply(results_osm.update)

for key, value in results_osm.most_common():
    print(f'Frecuencia: {value}, Palabra: {key} ')

Usando OSM se ven 538 casos no resueltos.
Podemos notar que las palabras "RUTA", "KM", "-", "ESQ", "SN", entre otras, presentan dificultad para OSM dada su alta frecuencia de aparición. Esto parece indicar que las direcciones problemáticas son aquellas sobre las rutas, en esquinas o sin número.
Otras palabras como "AV", "DE", "SAN", "CALLE" son realmente stopwords.



### Mapa de clientes

#### Geolocalizados con Google

In [ ]:
#conversion de latitud y longitud a mercator
k = 6378137
df['LONGITUDE_GOOGLE_MERCATOR']= df['LONGITUDE_GOOGLE'] * (k * np.pi/180.0)
df['LATITUDE__GOOGLE_MERCATOR'] = np.log(np.tan((90 + df['LATITUDE_GOOGLE']) * np.pi/360.0)) * k

In [ ]:
output_notebook()
tile_provider = get_provider(Vendors.CARTODBPOSITRON)

#ejes
p = figure(x_axis_type="mercator", y_axis_type="mercator")

#fondo de mapa
p.add_tile(tile_provider)

#puntos en el mapa
p.circle(y = df['LATITUDE__GOOGLE_MERCATOR'],
         x = df['LONGITUDE_GOOGLE_MERCATOR'],
         fill_color='orange',fill_alpha=.5)

#detalles del titulo
p.title.text = "Clientes geolocalizados con Google"
p.title.text_color = "Orange"
p.title.text_font_size = "25px"

show(p)

#### Geolocalizados con OpenStreetMap

In [ ]:
#conversion de latitud y longitud a mercator
k = 6378137
df['LONGITUDE_OSM_MERCATOR']= df['LONGITUDE_OSM'] * (k * np.pi/180.0)
df['LATITUDE__OSM_MERCATOR'] = np.log(np.tan((90 + df['LATITUDE_OSM']) * np.pi/360.0)) * k

In [ ]:
output_notebook()
tile_provider = get_provider(Vendors.CARTODBPOSITRON)

#ejes
p = figure(x_axis_type="mercator", y_axis_type="mercator")

#fondo de mapa
p.add_tile(tile_provider)

#puntos en el mapa
p.circle(y = df['LATITUDE__OSM_MERCATOR'],
         x = df['LONGITUDE_OSM_MERCATOR'],
         fill_color='orange',fill_alpha=.5)

#detalles del titulo
p.title.text = "Clientes geolocalizados con OpenStreetMap"
p.title.text_color = "Orange"
p.title.text_font_size = "25px"

show(p)

##Análisis de distancia entre geocodificación y coordenadas reales en 20 puntos

In [ ]:
#Se levanta el dataset con 20 coordenadas reales encontradas manualmente
df_check = pd.read_csv('coordinates_check.csv')
df_check.head(20)

In [ ]:
#Google: diferencias en metros versus dirección real
list_google = []
for i in range(20):
  loc_a = df_check.iloc[i]['PUNTO_GOOGLE']
  loc_b = df_check.iloc[i]['PUNTO_CHECK']
  list_google.append(round(distance.distance(loc_a, loc_b).m))
print(f'El error promedio de Google en estos 20 casos es de {round(sum(list_google) / float(len(list_google)))} metros')

In [ ]:
#OSM: diferencias en metros versus dirección real
list_osm = []
for i in range(20):
  loc_a = df_check.iloc[i]['PUNTO_OSM']
  loc_b = df_check.iloc[i]['PUNTO_CHECK']
  list_osm.append(round(distance.distance(loc_a, loc_b).m))
print(f'El error promedio de Google en estos 20 casos es de {round(sum(list_osm) / float(len(list_osm)))} metros')

In [ ]:
#diferencias en metros versus dirección real ordenadas por mayor distancia
df_differences = pd.DataFrame(list(zip(list_osm, list_google)), columns =['OSM','Google'])
df_differences['DIRECCION_CONCAT'] = df_check['DIRECCION_CONCAT']
df_differences.sort_values(by=['OSM'], ascending=False).head(20)

In [ ]:
#gráfico de barras de los errores
bar = df_differences.plot.bar(rot=0)
bar.set_ylabel('metros')
bar.set_xlabel('direccion #')
bar.set_title('Distancia versus punto real')

##Conclusiones

*   El servicio de Google resuelve casi la totalidad de las direcciones, mientras que OpenStreetMap falla en más de la mitad.
*   De las direcciones no resueltas, especialmente en OSM, se pueden identificar palabras que complican la geolocalización, tales como: RUTA, KM, ESQ, SN.
*   De las 20 direcciones chequeadas manualmente, Google tiene menor error que OSM en 18 casos. Además el error promedio de Google en estos 20 casos es de 36 metros mientras que el de OSM es de 913 metros.
*   Ya sea con Google o con OSM, siempre que sea posible es recomendable hacer un chequeo manual de las direcciones geolocalizadas.





